In [1]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import Tool
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

LLM setup

In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv("GEMINI_API_KEY")

# Now initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=api_key,
    temperature=0.0
)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Serpapi Setup (google search)

In [3]:
import os
serpapi_key = os.getenv("SERPAPI_KEY") 
os.environ["SERPAPI_API_KEY"] = serpapi_key 

Wikipedia search using build in tool

In [4]:
# Load the ReAct prompt
prompt = hub.pull("hwchase17/react")

# Create tools
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools = [
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="Useful for looking up information on Wikipedia"
    ),

]

# Create agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True
)

# Run the agent
result = agent_executor.invoke({
    "input": "When was Elon Musk born and what is his current age?"
})

print(result["output"])

/usr/local/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Action: Wikipedia
Action Input: Elon MuskPage: Elon Musk
Summary: Elon Reeve Musk (born June 28, 1971) is a businessman and entrepreneur known for his leadership of Tesla, SpaceX, Twitter, and xAI. Musk has been the wealthiest person in the world since 2021; as of October 2025, Forbes estimates his net worth to be around $500 billion.
Born into a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada; his Canadian citizenship is congenital, his mother having been born there. He received bachelor's degrees in 1997 from the University of Pennsylvania in Philadelphia, United States, before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002. That year, Musk also became an American citizen.
In 2002, Musk founded the space technology company

Google search

In [5]:
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import Tool, create_react_agent, AgentExecutor
from langchain import hub

# Load the ReAct prompt
prompt = hub.pull("hwchase17/react")

# Create SerpAPI wrapper
serpapi = SerpAPIWrapper()

# Create tools
tools = [
    Tool(
        name="Search",
        func=serpapi.run,
        description="Useful for searching current information on the internet, including real-time data like weather, AQI, news, etc."
    ),
]

# Create agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True
)

# Run the agent
result = agent_executor.invoke({
    "input": "What is the current AQI of Delhi? Give me just the number."
})

print(result["output"])

/usr/local/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Action: Search
Action Input: current AQI Delhi{'type': 'organic_result', 'title': 'Delhi Air Pollution: Real-time Air Quality Index (AQI)', 'source': 'aqicn.org', 'description': 'Delhi Air Pollution: Real-time Air Quality Index (AQI)'}Action: Search
Action Input: Delhi AQI number[{'title': 'Delhi AQI: Delhi AQI Hits 391, Air Quality Dips to ‘Severe’ Amid Dense Smog | NewsX', 'link': 'https://www.youtube.com/watch?v=CObFpqdDWzA', 'source': 'YouTube · NewsX Live', 'date': '5 hours ago', 'label': 'Video'}, {'title': 'Delhi air quality in ‘very poor’ category; no respite from pollution; AQI at 391', 'link': 'https://www.thehindu.com/news/cities/Delhi/delhi-air-quality-very-poor-aqi-weather/article70297510.ece', 'source': 'The Hindu', 'source_logo': 'https://serpapi.com/searches/691d9a31d0a88fdb7fe5432f/images/df9c515982fd108416531d9f66b88619d90c76cfb5640468391a3cd6f2fea768.png', 'date': '6 hours ago'}, {'title': 'Delhi air pollution: Delhiites wake u

In [6]:
prompt_temp = PromptTemplate(
    input_variables=["aqi"],
    template="What are the possible health dieases accociated with this aqi {aqi}",
)

chain = prompt_temp | llm | StrOutputParser()

res = chain.invoke({"aqi" : result})
print(res)

An AQI of 391 falls into the **"Hazardous"** category (301-500). At this level, the air quality is extremely poor and poses a serious health risk to **everyone**, not just sensitive groups.

Exposure to such high levels of pollution can lead to a wide range of immediate and long-term health problems. Here are the possible health diseases and conditions associated with an AQI of 391:

**Immediate/Short-Term Effects (can occur within hours or days):**

1.  **Severe Respiratory Symptoms:**
    *   **Asthma attacks:** Severe aggravation and increased frequency.
    *   **COPD exacerbations:** Worsening of chronic obstructive pulmonary disease symptoms.
    *   **Bronchitis:** Acute inflammation of the bronchial tubes.
    *   **Shortness of breath, coughing, wheezing, chest tightness:** Even in healthy individuals.
    *   **Increased susceptibility to respiratory infections:** Like pneumonia and common colds.
    *   **Reduced lung function:** Making it harder to breathe.

2.  **Cardiovas

Memory to a agent

In [12]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
convo = ConversationChain(
    llm=llm,
    # memory = ConversationBufferMemory(), # This will store the entire conversation
    memory = ConversationBufferWindowMemory(k=2), # This will store only the last k interactions
)

# First question
response1 = convo.predict(input="Who is the president of BCCI?")
print(f"1st RESPONSE : {response1}")

response2 = convo.predict(input="What the color of a banana?")
print(f"2nd RESPONSE : {response2}")

response3 = convo.predict(input="What is his age?")
print(f"3rd RESPONSE : {response3}")

response4 = convo.predict(input="What is his nationality?")
print(f"4th RESPONSE : {response4}")

Oh, that's a great question about Indian cricket! The current president of the **BCCI**, which stands for the **Board of Control for Cricket in India**, is **Mr. Roger Binny**.

He took over the prestigious role in **October 2022**, succeeding the former Indian captain, Sourav Ganguly. It was quite a significant moment because Roger Binny is a legendary figure in Indian cricket himself! He was a crucial all-rounder in the Indian team that famously won the **1983 Cricket World Cup** – a truly historic achievement for the nation.

The BCCI, just to give you a little more context, is the governing body for cricket in India and is actually the richest cricket board in the world. It was founded way back in **December 1928** and its headquarters are in **Mumbai, Maharashtra**. They oversee everything from organizing domestic tournaments like the Ranji Trophy and the Indian Premier League (IPL) to managing the Indian national cricket teams.

So, yes, Roger Binny, a former World Cup winner, is

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash
Please retry in 42.310674927s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 1

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash
Please retry in 39.892741958s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, retry_delay {
  seconds: 39
}
]